In [1]:
import pysam
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import itertools

In [2]:
bam_file = "/home/dbeb/btech/bb1160039/scratch/project/pbmc_10k_protein_v3_possorted_genome_bam.bam"
bai_file = "/home/dbeb/btech/bb1160039/scratch/project/pbmc_10k_protein_v3_possorted_genome_bam.bam.bai"

In [3]:
samf = pysam.Samfile(bam_file, "rb")
replicon_dict = dict([[replicon, {'seq_start_pos': 0,'seq_end_pos': length}] for replicon, length in zip(samf.references, samf.lengths)])

In [4]:
print(replicon_dict['1']['seq_start_pos'])
print(replicon_dict['1']['seq_end_pos'])

0
248956422


In [5]:
samfile = pysam.AlignmentFile(bam_file, "rb", index_filename = bai_file)

In [6]:
x=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','X','Y']
list_tags = []

for i in tqdm(range(0,len(x))):
    for read in samfile.fetch(x[i]):
        try:
            if read.has_tag("GX")==False and read.get_tag('NH')==1:
                list_tags.append([read.get_tag("CB"),read.get_tag("UB"),read.get_tag("GX")])
        except KeyError:
            continue

In [7]:
%%time
list_tags_rm_dup_final = list(list_tags for list_tags,_ in itertools.groupby(list_tags))

CPU times: user 25.9 s, sys: 1 s, total: 26.9 s
Wall time: 26.9 s


In [8]:
print(len(list_tags))
print(len(list_tags_rm_dup_final))

144894434
115424536


In [9]:
n=len(list_tags_rm_dup_final)
n

115424536

In [10]:
%%time
bins = 10
df_all = []
for i in range(bins):
    df_all.append(pd.DataFrame(list_tags_rm_dup_final[:int(n/bins)], columns=["celltag","moltag","pseudoname"]))

CPU times: user 52.4 s, sys: 2.42 s, total: 54.8 s
Wall time: 54.8 s


In [11]:
%%time
celltags = []
pseudonames = []
for i in range(bins):
    celltags.append(df_all[i]['celltag'].value_counts())
    pseudonames.append(df_all[i]['pseudoname'].value_counts())

CPU times: user 47.4 s, sys: 272 ms, total: 47.7 s
Wall time: 47.8 s


In [12]:
%time
df_all_subset = []
cutoff = 0
for i in range(bins):
    temp = df_all[i][(df_all[i]["celltag"].isin(celltags[i][celltags[i]>cutoff].index)) & 
                     (df_all[i]["pseudoname"].isin(pseudonames[i][pseudonames[i]>0].index))]
    df_all_subset.append(temp)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [13]:
%%time
counts_list = []
for i in range(bins):
    counts_list.append(df_all_subset[i].groupby(['pseudoname','celltag']).size().unstack('celltag', fill_value=0))

CPU times: user 49.6 s, sys: 9.95 s, total: 59.5 s
Wall time: 59.6 s


In [14]:
counts_list_columns = [counts_list[i].columns for i in range(bins)]
cell_bcode= sorted(set(counts_list_columns[0]).intersection(*counts_list_columns))

In [15]:
len(cell_bcode)

24600

In [16]:
%%time
for i in range(bins):
    counts_list[i] = counts_list[i][cell_bcode]

CPU times: user 6.48 s, sys: 961 ms, total: 7.44 s
Wall time: 7.44 s


In [17]:
%%time
counts_df = pd.concat(counts_list)

CPU times: user 942 ms, sys: 913 ms, total: 1.85 s
Wall time: 1.85 s


In [19]:
counts_df_final=counts_df.groupby(level=0, axis=0).sum()